In [1]:
from jupyter_innotater import *
import numpy as np, os

### Repeat - Multiple Bounding Boxes, Single Classification

At a per-photo level, allow classification stating whether the photo contains 'cats' or 'dogs'.

And within each photo allow up to 8 bounding boxes to be drawn around the individual animals.

To obtain the repeated bounding boxes we add an extra dimension to the target matrix and use the RepeatInnotation. This accepts configuration details on how to build new rows of the BoundingBoxInnotations - you do not instantiate BoundingBoxInnotations yourself, just tell it which class, data, and keyword params to use.

In [2]:
animalfns = sorted(os.listdir('./animals/'))

repeats = 8

# Per-photo data
classes = ['cat', 'dog']
targets_type = np.zeros((len(animalfns), 1), dtype='int') # Column vector containing class index

# Repeats within each photo
targets_bboxes = np.zeros((len(animalfns), repeats, 4), dtype='int') # (x,y,w,h) for each animal

Innotater( 
    ImageInnotation(animalfns, path='./animals', width=400, height=300), 
    [
        MultiClassInnotation(targets_type, name='Animal Type', classes=classes, dropdown=False),
        RepeatInnotation(
            (BoundingBoxInnotation, targets_bboxes),
             max_repeats=repeats, min_repeats=1
        )
    ]
)

Innotater(children=(HBox(children=(VBox(children=(ImagePad(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x0…

In [3]:
print('Class Indices', targets_type[:2]) # Just display the first 2 to save space
print('Bounding Boxes', targets_bboxes[:2])

Class Indices [[0]
 [1]]
Bounding Boxes [[[ 74   8 143 242]
  [174  12  92  95]
  [222  78 116 168]
  [  0   0   0   0]
  [  0   0   0   0]
  [  0   0   0   0]
  [  0   0   0   0]
  [  0   0   0   0]]

 [[  5  55 178 241]
  [215   4 185 249]
  [  0   0   0   0]
  [  0   0   0   0]
  [  0   0   0   0]
  [  0   0   0   0]
  [  0   0   0   0]
  [  0   0   0   0]]]


### Repeat - Multiple Bounding Boxes and Multiple Classification, Single BBox and Exclude checkbox

Per-photo we have an Exclude checkbox - perhaps to remove 'bad' photos from our dataset further down the line.

Per-photo there is a bounding box to be drawn around the main section of the image.

Also display the filename in a textbox beneath the image.

Within each photo draw bounding boxes and specify breed class of each individual animal, up to a maximum of 8.

In [4]:
repeats = 8

# Per-photo data
targets_exclude = np.zeros((len(animalfns), 1), dtype='int') # Binary flag to indicate want to exclude from dataset
targets_mainbbox = np.zeros((len(animalfns), 4), dtype='int') # (x,y,w,h) for each data row

# Repeats within each photo
breeds = ['Cat - Shorthair tabby', 'Cat - Shorthair ginger', 'Dog - Labrador', 'Dog - Beagle', 'Dog - Terrier']
targets_breed = np.zeros((len(animalfns), repeats, len(breeds)), dtype='int')
targets_breed[:,:,0] = 1 # Set up one-hot default to first class
targets_bboxes = np.zeros((len(animalfns), repeats, 4), dtype='int') # (x,y,w,h) for each animal

Innotater(
    [ 
        ImageInnotation(animalfns, path='./animals', width=370, height=280), # Display the image itself
        TextInnotation(animalfns, multiline=False) # Display the image filename
    ],
    [
        BinaryClassInnotation(targets_exclude, name='Exclude'), # Checkbox
        BoundingBoxInnotation(targets_mainbbox, name='Main Bounding Box'), # BBox for main image
        RepeatInnotation(
            (BoundingBoxInnotation, targets_bboxes), # Individual animal bounding box
            (MultiClassInnotation, targets_breed, 
                {'name':'Breed', 'classes':breeds, 'dropdown':True}), # Per-animal breed dropdown
            max_repeats=repeats, min_repeats=1
        )
    ]
)

Innotater(children=(HBox(children=(VBox(children=(ImagePad(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x0…

In [5]:
print('Exclude Flag', targets_exclude[:2]) # Just display the first 2 to save space
print('Main Bounding Boxes', targets_mainbbox[:2])
print('Animal Bounding Boxes', targets_bboxes[:2])
print('Animal Breeds', targets_breed[:2])

Exclude Flag [[0]
 [0]]
Main Bounding Boxes [[ 72   7 255 242]
 [  6   0 391 297]]
Animal Bounding Boxes [[[ 90   6 117 239]
  [191  15  86  85]
  [226  83 109 165]
  [  0   0   0   0]
  [  0   0   0   0]
  [  0   0   0   0]
  [  0   0   0   0]
  [  0   0   0   0]]

 [[ 12  44 171 250]
  [209   4 191 279]
  [  0   0   0   0]
  [  0   0   0   0]
  [  0   0   0   0]
  [  0   0   0   0]
  [  0   0   0   0]
  [  0   0   0   0]]]
Animal Breeds [[[1 0 0 0 0]
  [1 0 0 0 0]
  [1 0 0 0 0]
  [1 0 0 0 0]
  [1 0 0 0 0]
  [1 0 0 0 0]
  [1 0 0 0 0]
  [1 0 0 0 0]]

 [[0 0 0 1 0]
  [0 0 0 0 1]
  [1 0 0 0 0]
  [1 0 0 0 0]
  [1 0 0 0 0]
  [1 0 0 0 0]
  [1 0 0 0 0]
  [1 0 0 0 0]]]


### Grouped Innotations

The GroupedInnotation doesn't do any repeating but just allows you to group together other Innotations so they appear side-by-side.

In [6]:
targets_singlebb = np.zeros((len(animalfns), 4), dtype='int') # (x,y,w,h) for each data row
targets_cl = np.zeros((len(animalfns), 1), dtype='int')

Innotater( ImageInnotation(animalfns, path='./animals', width=370, height=280), 
           GroupedInnotation( # Just to place side-by-side
              MultiClassInnotation(targets_cl, name='Animal', classes=classes, dropdown=True),
              BoundingBoxInnotation(targets_singlebb)
           ) 
)

Innotater(children=(HBox(children=(VBox(children=(ImagePad(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x0…